In [1]:
import pandas as pd

import preprocessing as pp
import model
from train import train_model
from predict import predict

In [6]:
# Data loading
df_store = pd.read_csv('../data/store.csv')
df_train = pd.read_csv('../data/train.csv', low_memory=False)

## General pipeline test

### Model selection

In [8]:
# Preprocessing
X, y = pp.Preprocessor().transform(df_store, df_train)
#X = X.iloc[::-1]
#y = y.iloc[::-1]

X.head()

D:\Etudes\Cours\ING5\Modules\S2\ML Business Case\0. Repository\model\preprocessing.py:43: DataConversionWarning: Data with input dtype float32 were all converted to float64 by the scale function.
  df_store_bis = scale(df_store_bis)


,DayOfWeek,Open,Promo,SchoolHoliday,StateHoliday_a,StateHoliday_b,StateHoliday_c,cos_DayOfWeek,sin_DayOfWeek,1Component Store PCA,2Component Store PCA,3Component Store PCA,Date
1016094,3.0,1.0,0.0,1.0,0.0,0.0,0.0,-0.900969,4.338837e-01,-1.030289,0.102412,1.386819,2013-01-02
1016093,4.0,1.0,0.0,1.0,0.0,0.0,0.0,-0.900969,-4.338837e-01,-1.030289,0.102412,1.386819,2013-01-02
1016092,5.0,1.0,0.0,1.0,0.0,0.0,0.0,-0.222521,-9.749279e-01,-1.030289,0.102412,1.386819,2013-01-02
1016091,6.0,1.0,0.0,1.0,0.0,0.0,0.0,0.623490,-7.818315e-01,-1.030289,0.102412,1.386819,2013-01-02
1016090,7.0,0.0,0.0,1.0,0.0,0.0,0.0,1.000000,-2.449294e-16,-1.030289,0.102412,1.386819,2013-01-02


In [9]:
X.tail()

,DayOfWeek,Open,Promo,SchoolHoliday,StateHoliday_a,StateHoliday_b,StateHoliday_c,cos_DayOfWeek,sin_DayOfWeek,1Component Store PCA,2Component Store PCA,3Component Store PCA,Date
4,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.623490,0.781832,0.935551,-0.769879,-2.379503,2015-07-31
3,2.0,1.0,1.0,1.0,0.0,0.0,0.0,-0.222521,0.974928,0.935551,-0.769879,-2.379503,2015-07-31
2,3.0,1.0,1.0,1.0,0.0,0.0,0.0,-0.900969,0.433884,0.935551,-0.769879,-2.379503,2015-07-31
1,4.0,1.0,1.0,1.0,0.0,0.0,0.0,-0.900969,-0.433884,0.935551,-0.769879,-2.379503,2015-07-31
0,5.0,1.0,1.0,1.0,0.0,0.0,0.0,-0.222521,-0.974928,0.935551,-0.769879,-2.379503,2015-07-31


In [ ]:
reg_model = model.Regressor()
train_model(reg_model, X, y)

In [6]:
print(reg_model)

Regressor()
